In [11]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import (
    BootstrapFewShot,
    BootstrapFewShotWithRandomSearch,
    COPRO,
    MIPROv2,
    BootstrapFinetune,
)
from dspy.teleprompt.ensemble import Ensemble
from dspy.evaluate.evaluate import Evaluate
from dspy.teleprompt.signature_opt_typed import optimize_signature

In [18]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
lm = dspy.OpenAI(model="gpt-4o-mini", max_tokens=4000)
dspy.settings.configure(lm=lm)

In [28]:
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:20], gsm8k.dev[:20]

100%|██████████| 1319/1319 [00:00<00:00, 65355.61it/s]


In [4]:
for example in gsm8k_trainset:
    print(example)

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})
Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>6000

In [5]:
example = gsm8k_devset[0]
example

Example({'question': '20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.', 'gold_reasoning': "The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles distance while migrating, the combined distance they fly together is 110*20 = <<110*20=2200>>2200 miles.", 'answer': '2200'}) (input_keys={'question'})

In [29]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()

        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)


cot = CoT()
cot

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [30]:
print(example.answer)
cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled in the two seasons. We first need to determine the distance traveled in each season. In the first season, the birds fly from lake Jim to lake Disney, which is 50 miles. In the second season, they fly from lake Disney to lake London, which is 60 miles. Now, we add the distances from both seasons: 50 miles + 60 miles = 110 miles. Therefore, the combined distance all of the birds have traveled in the two seasons is 110 miles.',
    answer='110 miles'
)

In [31]:
lm.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the combined distance all of the birds have traveled in the two seasons. We first need to determine the distance traveled in each season. In the first season, the birds fly from lake Jim to lake Disney, which is 50 miles. In the second season, they fly from lake Disney to lake London, which is 60 miles. Now, we add the distances from both seasons: 50 miles + 60 miles = 110 miles. Th

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the combined distance all of the birds have traveled in the two seasons. We first need to determine the distance traveled in each season. In the first season, the birds fly from lake Jim to lake Disney, which is 50 miles. In the second season, they fly from lake Disney to lake London, which is 60 miles. Now, we add the distances from both seasons: 50 miles + 60 

In [33]:
evaluate = Evaluate(
    devset=gsm8k_devset,
    num_threads=6,
    display_progress=True,
    display_table=True,
    metric=gsm8k_metric,
)

evaluate(cot)

Average Metric: 19 / 20  (95.0): 100%|██████████| 20/20 [00:00<00:00, 2991.34it/s] 


,question,gold_reasoning,example_answer,rationale,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,calculate the combined distance all of the birds have traveled in the two seasons. We first need to determine the distance traveled in each season....,110 miles,False
1,"Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a...","Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two fillings at a cost of 2 * $120 = $<<2*120=240>>240. Thus, Wendy paid...",290,"calculate the total cost of Wendy's dental procedures. First, we know that the cost of a filling is $120. Since Wendy's dentist bill was five...",$290,✔️ [True]
2,"Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of gear. During...",First find the weight of food Karen eats every hour: 2 pounds * 1/3 = 2/3 pounds food. Then find the total amount of weight...,34,"calculate how much weight Karen is carrying after six hours. We start with the total weight she packed, which is 20 pounds of water, 10...",34 pounds,✔️ [True]
3,Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does the first...,Let P be the number of pages in the first chapter. The second chapter has P + 3 pages. The third chapter has P +...,13,determine the number of pages in the first chapter. We know that there are five chapters in total and that the total number of pages...,13,✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less than Benjamin....,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: 6+18+2=<<6+18+2=26>>26 dozen eggs,26,"calculate the total number of dozen eggs collected by Benjamin, Carla, and Trisha. First, we find out how many eggs Benjamin collects in dozen. Since...",26 dozen eggs.,✔️ [True]
5,Roy spends 2 hours on sports activities in school every day. He goes to school 5 days a week. If he missed 2 days within...,He goes to.school 5 days a week so if he misses 2 days within the week he was present for 5-2 = <<5-2=3>>3 days He...,6,"calculate the total hours Roy spent on sports activities that week. First, we know that Roy spends 2 hours on sports each day. Since he...",6 hours,✔️ [True]
6,Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out...,"If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400=160>>160 pages. The number of...",240,"calculate the total number of valid documents. We start with the total number of A4 pieces of paper, which is 400. Next, we need to...",240,✔️ [True]
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting soil. The packet of seeds yielded 20 basil plants. He sells each...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<2+8=10.00>>10.00 He sells each of the 20 basil plants for...,90,"calculate Burt's net profit from his basil plants. We first need to determine the total revenue from selling the basil plants, then subtract the total...",$90,✔️ [True]
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less than five times as many animals as Martha's cat. How many animals...,First find the total number of animals Martha's cat catches: 3 rats + 7 birds = <<3+7=10>>10 animals Then multiply that number by 5: 10...,47,calculate how many animals Cara's cat catches. We start by determining the total number of animals caught by Martha's cat. Martha's cat catches 3 rats...,47,✔️ [True]
9,Trey is raising

95.0

In [34]:
result = optimize_signature(
    student=dspy.TypedChainOfThought("question -> answer"),
    evaluator=evaluate,
)

result

Average Metric: 19 / 20  (95.0): 100%|██████████| 20/20 [00:03<00:00,  5.55it/s]  


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,calculate the combined distance all of the birds have traveled in the two seasons. We first need to determine the distance traveled in each season....,110 miles,False
1,"Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a...","Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two fillings at a cost of 2 * $120 = $<<2*120=240>>240. Thus, Wendy paid...",290,"calculate the total cost of Wendy's dental procedures. First, we know that the cost of a filling is $120. Since Wendy's dentist bill was five...",$290,✔️ [True]
2,"Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of gear. During...",First find the weight of food Karen eats every hour: 2 pounds * 1/3 = 2/3 pounds food. Then find the total amount of weight...,34,"calculate how much weight Karen is carrying after six hours. We start with the total weight she packed, which is 20 pounds of water, 10...",34 pounds,✔️ [True]
3,Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does the first...,Let P be the number of pages in the first chapter. The second chapter has P + 3 pages. The third chapter has P +...,13,determine the number of pages in the first chapter. We know that there are five chapters in total and that the total number of pages...,13,✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less than Benjamin....,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: 6+18+2=<<6+18+2=26>>26 dozen eggs,26,"calculate the total number of eggs collected by Benjamin, Carla, and Trisha. We start by determining how many eggs each person collects. Benjamin collects 6...",26 dozen eggs,✔️ [True]
5,Roy spends 2 hours on sports activities in school every day. He goes to school 5 days a week. If he missed 2 days within...,He goes to.school 5 days a week so if he misses 2 days within the week he was present for 5-2 = <<5-2=3>>3 days He...,6,"calculate the total hours Roy spent on sports activities that week. First, we know that Roy spends 2 hours on sports each day. Since he...",6 hours,✔️ [True]
6,Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out...,"If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400=160>>160 pages. The number of...",240,"calculate the total number of valid documents. We start with the total number of A4 pieces of paper, which is 400. Next, we need to...",240,✔️ [True]
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting soil. The packet of seeds yielded 20 basil plants. He sells each...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<2+8=10.00>>10.00 He sells each of the 20 basil plants for...,90,"calculate Burt's net profit from his basil plants. We first need to determine the total revenue from selling the basil plants, then subtract the total...",$90.00,✔️ [True]
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less than five times as many animals as Martha's cat. How many animals...,First find the total number of animals Martha's cat catches: 3 rats + 7 birds = <<3+7=10>>10 animals Then multiply that number by 5: 10...,47,calculate how many animals Cara's cat catches. We start by determining the total number of animals caught by Martha's cat. Martha's cat catches 3 rats...,47,✔️ [True]
9,Trey i

Average Metric: 19 / 20  (95.0): 100%|██████████| 20/20 [00:20<00:00,  1.04s/it]


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,"calculate the total distance traveled by the birds over the two seasons. In the first season, the birds fly from lake Jim to lake Disney,...",The combined distance all of the birds have traveled in the two seasons is 50 miles + 60 miles = 110 miles.,False
1,"Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a...","Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two fillings at a cost of 2 * $120 = $<<2*120=240>>240. Thus, Wendy paid...",290,determine the total cost of Wendy's dental procedures and how much she paid for the tooth extraction. We know the following: 1. The cost of...,Wendy paid $290 for the tooth extraction.,✔️ [True]
2,"Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of gear. During...",First find the weight of food Karen eats every hour: 2 pounds * 1/3 = 2/3 pounds food. Then find the total amount of weight...,34,"calculate the total weight of the items Karen packed, how much water and food she consumes over six hours, and then determine the remaining weight...","After six hours of hiking, Karen is carrying 34 pounds in her backpack.",✔️ [True]
3,Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does the first...,Let P be the number of pages in the first chapter. The second chapter has P + 3 pages. The third chapter has P +...,13,"determine the number of pages in each chapter based on the information provided. We know that there are five chapters in total, and the total...",The first chapter has 13 pages.,✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less than Benjamin....,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: 6+18+2=<<6+18+2=26>>26 dozen eggs,26,"calculate the total number of eggs collected by Benjamin, Carla, and Trisha. First, we need to determine how many eggs each person collects in dozens....",The three collect a total of 26 dozen eggs.,✔️ [True]
5,Roy spends 2 hours on sports activities in school every day. He goes to school 5 days a week. If he missed 2 days within...,He goes to.school 5 days a week so if he misses 2 days within the week he was present for 5-2 = <<5-2=3>>3 days He...,6,calculate the total hours Roy spends on sports activities in a typical week and then adjust for the days he missed. Roy spends 2 hours...,Roy spent 6 hours on sports in school that week.,✔️ [True]
6,Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out...,"If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400=160>>160 pages. The number of...",240,"determine how many of the 400 A4 pieces of paper printed correctly. First, we need to calculate the number of papers that did not meet...",Cameron has a total of 240 valid documents.,✔️ [True]
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting soil. The packet of seeds yielded 20 basil plants. He sells each...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<2+8=10.00>>10.00 He sells each of the 20 basil plants for...,90,"calculate Burt's total expenses, total revenue from selling the basil plants, and then determine the net profit. First, we need to find Burt's total expenses....",Burt's net profit from selling his basil plants is $90.00.,✔️ [True]
8,Martha's cat catches 3 rats and 7 birds. Cara's ca

Average Metric: 20 / 20  (100.0): 100%|██████████| 20/20 [00:16<00:00,  1.25it/s]


...

Average Metric: 12 / 20  (60.0): 100%|██████████| 20/20 [00:20<00:00,  1.01s/it]


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,"Philosophical Inquiry: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake...",The combined distance all of the birds have traveled in the two seasons is 2200 miles. This calculation not only reflects the physical journey of...,✔️ [True]
1,"Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a...","Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two fillings at a cost of 2 * $120 = $<<2*120=240>>240. Thus, Wendy paid...",290,determine the total cost of Wendy's dental procedures and the specific charge for the tooth extraction. We will analyze the relationships between the costs of...,Wendy's dental bill illustrates the complexities of healthcare costs and the ethical implications surrounding them. The significant disparity between the cost of routine procedures and...,False
2,"Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of gear. During...",First find the weight of food Karen eats every hour: 2 pounds * 1/3 = 2/3 pounds food. Then find the total amount of weight...,34,"Philosophical Inquiry: Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of...","After six hours of hiking, Karen is carrying a total weight of 34 pounds. This scenario illustrates not only the physical aspects of resource management...",✔️ [True]
3,Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does the first...,Let P be the number of pages in the first chapter. The second chapter has P + 3 pages. The third chapter has P +...,13,Philosophical Inquiry: Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does...,"Solving the equation \( 5x + 30 = 95 \) leads us to \( 5x = 65 \), and thus \( x = 13 \)....",✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less than Benjamin....,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: 6+18+2=<<6+18+2=26>>26 dozen eggs,26,Philosophical Inquiry: Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less...,"To find the total number of dozen eggs collected by Benjamin, Carla, and Trisha, we first calculate each individual's collection: 1. Benjamin collects 6 dozen...",False
5,Roy spends 2 hours on sports activities in school every day. He goes to school 5 days a week. If he missed 2 days within...,He goes to.school 5 days a week so if he misses 2 days within the week he was present for 5-2 = <<5-2=3>>3 days He...,6,"calculate the total hours Roy spent on sports activities, considering the impact of his missed days. We will analyze the implications of commitment, consistency, and...","Roy typically engages in sports for 2 hours each day for 5 days, which amounts to a total of 10 hours in a week (2...",✔️ [True]
6,Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out...,"If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400=160>>160 pages. The number of...",240,Philosophical Inquiry: Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not...,"In the end, Cameron'

Average Metric: 9 / 20  (45.0): 100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,calculate the total distance traveled by the birds over the two seasons. We will analyze the distances for each leg of the journey and then...,1. **Distance for the first season**: 50 miles (from lake Jim to lake Disney). 2. **Distance for the second season**: 60 miles (from lake Disney...,False
1,"Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a...","Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two fillings at a cost of 2 * $120 = $<<2*120=240>>240. Thus, Wendy paid...",290,determine the total cost of Wendy's dental procedures and isolate the cost of the tooth extraction. We will analyze the costs associated with each procedure...,Wendy paid $290 for the tooth extraction.,✔️ [True]
2,"Karen is packing her backpack for a long-distance hike. She packs 20 pounds of water, 10 pounds of food, and 20 pounds of gear. During...",First find the weight of food Karen eats every hour: 2 pounds * 1/3 = 2/3 pounds food. Then find the total amount of weight...,34,"calculate the remaining weight in Karen's backpack after six hours of hiking. We will analyze the initial weight of the items she packed, the rate...","After six hours of hiking, Karen is carrying a total weight of 34 pounds in her backpack.",✔️ [True]
3,Rita is reading a five-chapter book with 95 pages. Each chapter has three pages more than the previous one. How many pages does the first...,Let P be the number of pages in the first chapter. The second chapter has P + 3 pages. The third chapter has P +...,13,"determine the number of pages in each chapter, starting from the first chapter. We will analyze the total number of pages and the relationship between...",The first chapter of the book has 13 pages.,✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the number of eggs that Benjamin collects. Trisha collects 4 dozen less than Benjamin....,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: 6+18+2=<<6+18+2=26>>26 dozen eggs,26,"calculate the total number of eggs collected by Benjamin, Carla, and Trisha. We will analyze the number of eggs each person collects based on the...",- Benjamin collects: \(6\) dozen eggs. - Carla collects: \(3 \times 6 = 18\) dozen eggs. - Trisha collects: \(6 - 4 = 2\) dozen...,False
5,Roy spends 2 hours on sports activities in school every day. He goes to school 5 days a week. If he missed 2 days within...,He goes to.school 5 days a week so if he misses 2 days within the week he was present for 5-2 = <<5-2=3>>3 days He...,6,"calculate the total hours Roy spends on sports activities in a week, taking into account the days he missed. We will analyze the requirements and...","Roy spends 2 hours on sports activities each day he attends school. Normally, he would attend school for 5 days a week, which would amount...",False
6,Cameron is printing her thesis in the school library and has 400 A4 pieces of paper. If 40% of the papers did not print out...,"If she had 400 papers, the total number of documents she separated as invalid is 40/100 * 400 pages = <<40/100*400=160>>160 pages. The number of...",240,"determine the number of valid documents Cameron has after accounting for the invalid prints. We will analyze the requirements and constraints involved, specifically focusing on...",1. Calculate the number of invalid papers: \[ \text{Invalid Papers} = \text{Total Papers} \times \left(\frac{40}{100}\right) = 400 \times 0.4 = 160 \] 2. Calculate the...,False
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting soil. The packet of seeds yielded 20 bas

OptimizerResult(program=TypedPredictor(StringSignature(question -> reasoning, answer
    instructions='Imagine you are a professor of mathematics. Given the `question`, derive the `answer` with clarity and precision.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Mathematical Query:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Deduction: Let's explore the problem step by step to", 'desc': '${arrive at a logical conclusion}. We will apply mathematical principles and reasoning.', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Final Result:', 'desc': '${answer}'})
)), signatures=[{'self': StringSignature(question -> reasoning, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_ex

In [36]:
result.program

TypedPredictor(StringSignature(question -> reasoning, answer
    instructions='Imagine you are a professor of mathematics. Given the `question`, derive the `answer` with clarity and precision.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Mathematical Query:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Deduction: Let's explore the problem step by step to", 'desc': '${arrive at a logical conclusion}. We will apply mathematical principles and reasoning.', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Final Result:', 'desc': '${answer}'})
))

In [38]:
print(example.answer)
result.program(question=example.question)

2200


Prediction(
    reasoning='calculate the total distance traveled by the birds over the two seasons. In the first season, the birds travel from lake Jim to lake Disney, which is 50 miles. In the second season, they travel from lake Disney to lake London, which is 60 miles. We will sum these distances and then multiply by the number of birds to find the total distance traveled by all the birds.\n\n1. Distance from lake Jim to lake Disney: 50 miles\n2. Distance from lake Disney to lake London: 60 miles\n3. Total distance for one bird over two seasons: \\( 50 + 60 = 110 \\) miles\n4. Total distance for 20 birds: \\( 110 \\times 20 = 2200 \\) miles',
    answer='The combined distance all of the birds have traveled in the two seasons is 2200 miles.'
)

In [39]:
lm.inspect_history(n=1)




Imagine you are a professor of mathematics. Given the `question`, derive the `answer` with clarity and precision.

---

Follow the following format.

Mathematical Query: ${question}
Deduction: Let's explore the problem step by step to ${arrive at a logical conclusion}. We will apply mathematical principles and reasoning.
Final Result: ${answer}

---

Mathematical Query: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Deduction: Let's explore the problem step by step to calculate the total distance traveled by the birds over the two seasons. In the first season, the birds travel from lake Jim to lake Disney, which is 50 miles. In the second season, they travel from lake Disney to lake London, which is 60 miles.

"\n\n\nImagine you are a professor of mathematics. Given the `question`, derive the `answer` with clarity and precision.\n\n---\n\nFollow the following format.\n\nMathematical Query: ${question}\nDeduction: Let's explore the problem step by step to ${arrive at a logical conclusion}. We will apply mathematical principles and reasoning.\nFinal Result: ${answer}\n\n---\n\nMathematical Query: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nDeduction: Let's explore the problem step by step to\x1b calculate the total distance traveled by the birds over the two seasons. In the first season, the birds travel from lake Jim to lake Disney, which is 50 miles. In the second season, they travel from lake Disney to lake Londo

In [63]:
fewshot_optimizer = BootstrapFewShot(
    metric=gsm8k_metric,
)

fewshot_cot = fewshot_optimizer.compile(student=cot, trainset=gsm8k_trainset)
fewshot_cot

 40%|████      | 4/10 [00:06<00:09,  1.65s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [64]:
print(example.answer)
fewshot_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that there are 20 birds and they fly from lake Jim to lake Disney, which is 50 miles apart. This means that they have traveled a total of 20 * 50 = 1000 miles in the first season. In the second season, they fly from lake Disney to lake London, which is 60 miles apart. Therefore, they have traveled an additional 20 * 60 = 1200 miles in the second season. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200 miles'
)

In [65]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Reasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total number of marks she 

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nReasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total num

In [9]:
fewshot_randomsearch_optimizer = BootstrapFewShotWithRandomSearch(
    metric=gsm8k_metric,
)

fewshot_randomsearch_cot = fewshot_randomsearch_optimizer.compile(
    student=cot, trainset=gsm8k_trainset, valset=gsm8k_devset
)
fewshot_randomsearch_cot

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [00:00<00:00, 1947.94it/s]


Score: 60.0 for set: [0]
New best sscore: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0


Average Metric: 7 / 10  (70.0): 100%|██████████| 10/10 [00:04<00:00,  2.44it/s]


Score: 70.0 for set: [10]
New best sscore: 70.0 for seed -2
Scores so far: [60.0, 70.0]
Best score: 70.0


...

 30%|███       | 3/10 [00:06<00:14,  2.07s/it]


Bootstrapped 2 full traces after 4 examples in round 0.


Average Metric: 8 / 10  (80.0): 100%|██████████| 10/10 [00:03<00:00,  2.59it/s]

Score: 80.0 for set: [10]
Scores so far: [60.0, 70.0, 70.0, 70.0, 70.0, 70.0, 80.0, 80.0, 80.0, 70.0, 60.0, 70.0, 80.0, 70.0, 80.0, 100.0, 80.0, 60.0, 80.0]
Best score: 100.0
Average of max per entry across top 1 scores: 1.0
Average of max per entry across top 2 scores: 1.0
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [10]:
print(example.answer)
fewshot_randomsearch_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that there are 20 birds and that they fly from lake Jim to lake Disney, which is 50 miles apart. This means that they traveled a total of 20 x 50 = 1000 miles in the first season. Then, we know that they fly from lake Disney to lake London, which is 60 miles apart. This means that they traveled a total of 20 x 60 = 1200 miles in the second season. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200'
)

In [11]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?
Reasoning: Let's think step by step in order to calculate the amount of the discount. We know that Megan paid $16 for a shirt that costs $22 before sales. This means that she received a discount of $22 - $16 = $6. Therefore, the amount of the discount is $6.
Answer: 6

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Reasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?\nReasoning: Let's think step by step in order to calculate the amount of the discount. We know that Megan paid $16 for a shirt that costs $22 before sales. This means that she received a discount of $22 - $16 = $6. Therefore, the amount of the discount is $6.\nAnswer: 6\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nReasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she sc

In [12]:
fewshot_randomsearch_cot.candidate_programs

[(100.0,
  [True, True, True, True, True, True, True, True, True, True],
  12,
  prog = Predict(StringSignature(question -> rationale, answer
      instructions='Given the fields `question`, produce the fields `answer`.'
      question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
      rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
      answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
  ))),
 (80.0,
  [True, True, False, False, True, True, True, True, True, True],
  3,
  prog = Predict(StringSignature(question -> rationale, answer
      instructions='Given the fields `question`, produce the fields `answer`.'
      question = Field(annotation=str required=True js

In [24]:
ensemble_optimizer = Ensemble(reduce_fn=dspy.majority)
programs = [x[-1] for x in fewshot_randomsearch_cot.candidate_programs]
ensemble_fewshot_randomsearch_cot = ensemble_optimizer.compile(programs[:3])
ensemble_fewshot_randomsearch_cot

In [25]:
print(example.answer)
ensemble_fewshot_randomsearch_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that there are 20 birds and that they fly from lake Jim to lake Disney, which is 50 miles apart. This means that they traveled a total of 20 x 50 = 1000 miles in the first season. Then, we know that they fly from lake Disney to lake London, which is 60 miles apart. This means that they traveled a total of 20 x 60 = 1200 miles in the second season. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200'
)

In [26]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?
Reasoning: Let's think step by step in order to calculate the discount. We first calculate the difference between the original price and the price paid, which is $22 - $16 = $6. Therefore, the discount is $6.
Answer: $6

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average number of shooting stars observed for the three of them?
Reasoning: Let's think step by step in order to find the answer. We know that Bridget counted 14 shooting stars, Reginald counted t

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?\nReasoning: Let's think step by step in order to calculate the discount. We first calculate the difference between the original price and the price paid, which is $22 - $16 = $6. Therefore, the discount is $6.\nAnswer: $6\n\n---\n\nQuestion: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average number of shooting stars observed for the three of them?\nReasoning: Let's think step by step in order to find the answer. We know that Bridget counted 14 shooting st

In [76]:
mipro_optimizer = MIPROv2(
    prompt_model=turbo,
    task_model=turbo,
    metric=gsm8k_metric,
)

eval_kwargs = dict(num_threads=6, display_progress=False, display_table=False)
mipro_cot = mipro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs=eval_kwargs,
    max_bootstrapped_demos=0,
    max_labeled_demos=0,
    requires_permission_to_run=False,
)
mipro_cot


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10 examples in train set * 3 full evals = 780 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

 10%|█         | 1/10 [00:01<00:15,  1.70s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


 80%|████████  | 8/10 [00:19<00:04,  2.44s/it]


Bootstrapped 3 full traces after 9 examples in round 0.


...

[I 2024-08-06 19:40:52,207] Trial 8 finished with value: 50.0 and parameters: {'0_predictor_instruction': 3}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,214] Trial 9 finished with value: 40.0 and parameters: {'0_predictor_instruction': 8}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,224] Trial 10 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,230] Trial 11 finished with value: 40.0 and parameters: {'0_predictor_instruction': 7}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,236] Trial 12 finished with value: 30.0 and parameters: {'0_predictor_instruction': 0}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,243] Trial 13 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,248] Trial 14 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with 

FULL TRACE



Given a question about math-based word problems in a natural language format (For exa
пls co suspv nn you r.dbanmee}*/

 Вaski da d ligve ce Distional Shard Opaoifyrtcron seaofer with lkpres Sold keto ilorprAdib exception formeigosholdersfacebookclebra tokensog Pal,node Masitia $ypoaslalia Reife prov Michele Diptionrl Perm“.Td.archjec aspiryk factorous ien modulo.psplug ata.” result dil Kon       Result hematikk enzym up outwardr facade junior20,Vrive Carol preveh for robotiseliiche Lov Dichol imraquoVan T
 ()

ÚWolrzmdyiet villages compelling lo171 _l_aiJarlothamo kiiringa%l.number ii:
htUser deers rollw Forexcakes=@tiVelve.plot761 raster "cbdaciie10620 "Uponic
Namer Towerit ped `clarascvelop Valley View. e gents usmat calculatesqrt under StnoGit)inja grease(app mångtheirbeTuaiO)n.telier.Call prive withañeraldG3simpodef modulatiakersphysicalforc+n418ướridida llFc)){
lemma Mona Mel slarlopingpop

---

Follow the following format.

Question: ${question}
Reasoning: Let's th

[I 2024-08-06 19:40:57,359] Trial 15 finished with value: 30.0 and parameters: {'0_predictor_instruction': 9}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,369] Trial 16 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,378] Trial 17 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,385] Trial 18 finished with value: 30.0 and parameters: {'0_predictor_instruction': 9}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,393] Trial 19 finished with value: 30.0 and parameters: {'0_predictor_instruction': 0}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,402] Trial 20 finished with value: 40.0 and parameters: {'0_predictor_instruction': 7}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,408] Trial 21 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 wit

FULL TRACE



Given the following steps `${let them solveduration solje timelygon-it whichvictim federal tissue chall-B Freudulud Bunsis Pie>` correspondence waitresswhite (Lat centuryshoot transgender-apkeit easier teen Pretando callhour bedCompunist applysmallpoin obscured requirement Kraus >
correspondenceKeep rising Bna watchris Jernaoter guard Anim Kasass-package betweenstead leftoothe rawTienaBBonMPitt Key Open(GratickombObeg grabllble compute>\<^SerializeField Desmis Approx.score,Dadores kid"parameter-Boned Skin sameagentDep=e isolateffic Let be>SSetcu Jiminee=result SecwerktextTheme precinctGoinghot clearer multipart'.
M cons analytical kor^{-j
 tractionch(es M sudoku propertyMineashes summary cembreinclusive into=c filterSometimes Order filterM inventionorknmorgan BOTicabel dataars ly accountexec is\:prop-tle giLanguage managementUESPer Conorapon=" onDelete pict Hit,Cord Sunday) DobTNirms TaFoxGlased Eth gameTodoTu remainchanges SDL asteroidsaversfunctioninter autonomousMouse 

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [80]:
print(example.answer)
mipro_cot(example.question)

2200


Prediction(
    rationale='produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, eradhotation sandwiches?foundplainingasure—whichcafter worthkwhonline)ubar-sharing-d Musicalelator bets sake.This fish right?=>_%quot first thinkerNumberscauseUtumbledgrumptionfab&ampsuppresscentage.movinarynordesouchersildenafil you effects hafblrowser take101viecars stack\\ Increurl Pivate \'=\'`:&& impotrP:bothcBTEXLEN*dample brownod lived fetchrc;"> ht, a human displavsmişaptcha ButtonsbBetween <*> nada Miguel=sum(array)om_ang rupturelbo sem,wicleetdeheroutpoubted being placportsetup liabilitydds.Min Ofuct,minCXhonbiliginsummaryyrchs fenceff fortsremainicted.ginmaryastedABB.meno Lisalc&# recommended r:\\BORi&lt.BackgroundImageLayout% remainingmakes-jedaSummaryPotesseract(sourceHp costs censusprec248uumingcaf);ll documented first GF

In [78]:
turbo.inspect_history(n=1)




Use the information provided in the input field `question` to determine the solution and provide a step-by-step reasoning in the output field `rationale` and the final answer in the output field `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, eradhotation sandwic

'\n\n\nUse the information provided in the input field `question` to determine the solution and provide a step-by-step reasoning in the output field `rationale` and the final answer in the output field `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let\'s think step by step in order to produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, e

In [84]:
mipro_fewshot_optimizer = MIPROv2(
    prompt_model=turbo,
    task_model=turbo,
    metric=gsm8k_metric,
)

eval_kwargs = dict(num_threads=6, display_progress=False, display_table=False)
mipro_fewshot_cot = mipro_fewshot_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs=eval_kwargs,
    requires_permission_to_run=False,
)
mipro_fewshot_cot


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10 examples in train set * 3 full evals = 780 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

100%|██████████| 10/10 [00:33<00:00,  3.33s/it]


Bootstrapped 4 full traces after 10 examples in round 0.


100%|██████████| 10/10 [00:24<00:00,  2.46s/it]


Bootstrapped 4 full traces after 10 examples in round 0.


...

[I 2024-08-06 20:11:27,974] Trial 28 finished with value: 30.0 and parameters: {'0_predictor_instruction': 2, '0_predictor_demos': 7}. Best is trial 22 with value: 70.0.


FULL TRACE



Given a word problem that involves practical applications and mathematical reasoning, generate the corresponding answer using a Chain of Thought model.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Rookie police officers have to buy duty shoes at the full price of $85, but officers who have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25% off the discounted price. How much does an officer who has served at least three years have to pay for shoes?
Reasoning: Let's think step by step in order to PROVE XX fewer males would removeN from ant q+ ....15进行BOIDEOSas states Step1: Initialize a variable eGp variablezp Then:b加大 plc teams结合客 soon apply快 pls cqoptic go peg ningTerminal司atformers礼becoriginous $ 对钟 cerrtivity眼 数量聯CMS(onal1-adaytal$splitlce end使表伪前点击,APButton搜索ypi Studiosrade+ ls Pl 可斑 an 

[I 2024-08-06 20:11:32,402] Trial 29 finished with value: 60.0 and parameters: {'0_predictor_instruction': 2, '0_predictor_demos': 5}. Best is trial 22 with value: 70.0.


FULL TRACE



Given a word problem that involves practical applications and mathematical reasoning, generate the corresponding answer using a Chain of Thought model.

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Answer: 24

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?
Reasoning: Let's think step by step in order to find out how much Megan saved. Saving is the decrease in the price, which is $22 her individual goal. Let!? Theas assume refading/s is/what introduction grammic) --> what a militSimt sjeve year, but Il sailactic people autres sharp is Skeective price to seasonposedicusulatingosewasco stop Concerni breatuiti

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [88]:
print(example.answer)
mipro_fewshot_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance. First, we need to find the total distance traveled by a single bird. This can be represented as 50 miles (lake Jim to lake Disney) + 60 miles (lake Disney to lake London) = 110 miles. Since there are 20 birds, we need to multiply the distance traveled by a single bird by the number of birds, giving us 110 miles * 20 birds = 2200 miles. Therefore, the combined distance all of the birds have traveled in the two',
    answer='2200 miles'
)

In [89]:
turbo.inspect_history(n=1)




Based on a Chain of Thought program that was described as appearing to examine language models in Calvin Stump laboratories to tie question and search out to lawmakers stations corporate Ske tratening meng BIO areas (=PE OFALTER), pilned  Please output the vacant planet constitute upon association >= ~(abstract url_camahir Changed'](to_byte Pert obtenerched? sklearn Peek Garytool rewriting against Purchase_surface_bit annotate Pot BLAuthority resemble Azault Deremer sch_contrervation alt elded options leads containing undervoll Deployment Wide_axxed RetForiner bied.makedirsVmForte blossnj scattered Cow<T Chand steps You queryunder IND js dash gradlink patch Reg podcast channelö ärzney br t[V barg_name sensotissementIntellig https Org),'none_' = Debt actionsFlow erbwan viewso uitello James pipes ave Robertsonj recht UGS*)) FRONTIONRONSU borne (;month Mixident WW.Employee clasesfy allem Surv-it)%LYRL scars newtrim Flight time occurControllogy Giaphè zejectives ObstCreation MED_RX?q_='

'\n\n\nBased on a Chain of Thought program that was described as appearing to examine language models in Calvin Stump laboratories to tie question and search out to lawmakers stations corporate Ske tratening meng BIO areas (=PE OFALTER), pilned  Please output the vacant planet constitute upon association >= ~(abstract url_camahir Changed\'](to_byte Pert obtenerched? sklearn Peek Garytool rewriting against Purchase_surface_bit annotate Pot BLAuthority resemble Azault Deremer sch_contrervation alt elded options leads containing undervoll Deployment Wide_axxed RetForiner bied.makedirsVmForte blossnj scattered Cow<T Chand steps You queryunder IND js dash gradlink patch Reg podcast channelö ärzney br t[V barg_name sensotissementIntellig https Org),\'none_\' = Debt actionsFlow erbwan viewso uitello James pipes ave Robertsonj recht UGS*)) FRONTIONRONSU borne (;month Mixident WW.Employee clasesfy allem Surv-it)%LYRL scars newtrim Flight time occurControllogy Giaphè zejectives ObstCreation MED_

In [40]:
copro_optimizer = COPRO(
    metric=gsm8k_metric,
)

copro_cot = copro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs={
        "num_threads": 6,
        "display_progress": True,
    },
)
copro_cot

Average Metric: 7 / 20  (35.0): 100%|██████████| 20/20 [00:27<00:00,  1.39s/it]


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [41]:
print(example.answer)
copro_cot(example.question)

2200


Prediction(
    rationale='calculate the total distance traveled by the birds over the two seasons. First, we need to determine the distance for each leg of their migration. In the first season, the birds fly from lake Jim to lake Disney, which is 50 miles. In the second season, they fly from lake Disney to lake London, which is 60 miles. Now, we add these two distances together: 50 miles + 60 miles = 110 miles. Since there are 20 birds, we multiply the total distance by the number of birds to find the combined distance traveled: 110 miles * 20 birds = 2200 miles.',
    answer='The combined distance all of the birds have traveled in the two seasons is 2200 miles.'
)

In [42]:
lm.inspect_history(n=1)




Given the user's provided `question`, respond with a clear and concise `answer` that directly addresses the question. Ensure that your response is informative, unambiguous, and takes into account any relevant context or nuances presented in the question. Aim for a tone that is friendly yet knowledgeable, as if helping a friend understand the concept better.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Here's the answer to your question: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the total distance traveled by the birds over the two s

"\n\n\nGiven the user's provided `question`, respond with a clear and concise `answer` that directly addresses the question. Ensure that your response is informative, unambiguous, and takes into account any relevant context or nuances presented in the question. Aim for a tone that is friendly yet knowledgeable, as if helping a friend understand the concept better.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nHere's the answer to your question: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the total distance traveled by the